In [ ]:
!pip install matplotlib

In [ ]:
import syft as sy
%matplotlib inline
duet = sy.join_duet(loopback=True)

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.lin =self.torch_ref.nn.Linear(6,10)
    def forward(self, x):
        x = self.lin(x)
        return x

In [ ]:
import torch
model1 = SyNet(torch)
model1_ptr = model1.send(duet)

In [ ]:
opt1 = duet.torch.optim.SGD(params=model1_ptr.parameters(),lr=0.00001)

In [ ]:
duet.store.pandas

In [ ]:
data_pointer = duet.store[0]

In [ ]:
import numpy as np

In [ ]:

model2 = torch.nn.Linear(10,6)
opt2 = torch.optim.SGD(params=model2.parameters(), lr=0.00001)

target = torch.tensor([[19.77,20.34,20.5,20.62,20.73,20.89],[20.96,21.57,21.68,22.15,23.22,24.59],
                       [24.63,24.71,24.88,24.96,25.61,26.78],[27.59,27.85,28.97,36.25,37.19,37.33],
                       [37.48,37.56,38.04,39.28,39.33,39.46],[40.27,40.43,40.66,41.77,41.83,41.99],
                       [42.51,42.6,42.75,42.86,42.94,43.16],[43.22,43.49,43.94,44.17,45.94,48.38],
                       [48.66,52.14,55.33,57.65,61.15,65.09],[68.45,69.5,70.49,71.32,72.41,73.16]])
target.shape
#target.type

In [ ]:
%%time
import matplotlib.pyplot as plt
#epoch = 200
losses = []
predictions = []
for iter in range(250):
    opt1.zero_grad()
    opt2.zero_grad()
    
    activation_ptr = model1_ptr(data_pointer)
    activation = activation_ptr.clone().get(request_block=True)
    
    pred = model2(activation)
    
    #target = target.detach().numpy()
    loss = ((pred - target)**2).sum()
   
    loss.backward()
    
    grad_ptr = activation.grad.clone().send(duet)
    activation_ptr.backward(grad_ptr)
    
    opt1.step()
    opt2.step()
    losses.append(loss.tolist())
    predictions.append(pred.tolist())
    
    print("losses: ",losses)
    print('Predictions =',pred)


In [ ]:
%%time
import matplotlib.pyplot as plt

plt.title('Diameter of Bulb-AG Hydroponics')
plt.plot(losses, color='pink')
plt.ylabel("Loss")
plt.xlabel("epoch")
plt.show()